In [10]:
import sys
sys.path.append("../")
from inference.dan import DANv5

dan = DANv5(model_path="../weights/dan_b2.engine")

⚙️ Loading TensorRT inference engine ../weights/dan_b2.engine
⚙️ TensorRT engine loaded successfully.
 - 📥 Input images_a: shape (2, 3, 1280, 1280), dtype uint8
 - 📥 Input images_b: shape (2, 3, 640, 640), dtype uint8
 - 📤 Output output_a0: shape (2, 102000, 14), dtype float32
 - 📤 Output output_a1: shape (2, 500), dtype float32
 - 📤 Output output_a2: shape (2, 500), dtype float32
 - 📤 Output output_b0: shape (2, 25200, 14), dtype float32
 - 📤 Output output_b1: shape (2, 500), dtype float32
 - 📤 Output output_b2: shape (2, 500), dtype float32
⏳ Warming up...


In [11]:
from urllib.request import urlopen
import numpy as np
from PIL import Image
from utils.horizon import draw_horizon, draw_bboxes

url_rgb = "https://github.com/SEA-AI/.github/blob/main/assets/example_1_RGB.jpg?raw=true"
url_ir = "https://github.com/SEA-AI/.github/blob/main/assets/example_1_IR.jpg?raw=true"

image_rgb = np.array(Image.open(urlopen(url_rgb)).convert("RGB"))
image_ir = np.array(Image.open(urlopen(url_ir)).convert("RGB"))

image_rgb = np.stack([image_rgb for _ in range(dan.model.input_bs[0])], axis=0)
image_ir = np.stack([image_ir for _ in range(dan.model.input_bs[1])], axis=0)

dan((image_rgb, image_ir))
n = 100
for i in range(n):
    preds_rgb, preds_ir = dan((image_rgb, image_ir))

# print average times
for name, profile in dan.profiles.items():
    print(f"{profile.t / n * 1E3:>5.1f} ms [avg] - {name}")

preds = preds_rgb[0]
image = image_rgb[0]


def draw_preds(image, preds):
    I = image.copy()
    bboxes = preds[:, :4]  # x1, y1, x2, y2
    scores = preds[:, 4]  # scores (probabilities)
    classes = preds[:, 5]  # indices of classes

    hor_points, hor_score = bboxes[classes == -1], scores[classes == -1]
    bboxes, scores, classes = (
        bboxes[classes != -1],
        scores[classes != -1],
        classes[classes != -1],
    )

    dets = np.hstack((bboxes, scores[:, np.newaxis], classes[:, np.newaxis]))
    I = draw_bboxes(I, dets, color=(0, 0, 255), thickness=1)
    I = draw_horizon(
        I, keypoints=hor_points.reshape(-1, 2), color=(0, 255, 0), diameter=1
    )
    return Image.fromarray(I)

  3.4 ms [avg] - preprocess
 15.2 ms [avg] - inference
  6.8 ms [avg] - postprocess


In [12]:
draw_preds(image_rgb[0], preds_rgb[0])

In [13]:
draw_preds(image_ir[0], preds_ir[0])

In [14]:
# inference
dan.detect((image_rgb, image_ir), output_mode="tf")

[[{'detection_boxes': array([[0.42056396, 0.60360718, 0.42693604, 0.61279907],
          [0.50881348, 0.53591614, 0.52868652, 0.54377136],
          [0.51214844, 0.52104492, 0.56160156, 0.56176758],
          [0.38707717, 0.        , 0.45749426, 1.        ]]),
   'detection_scores': array([0.62597656, 0.23095703, 0.8125    , 0.27319533]),
   'detection_classes': array([ 2.,  1.,  4., -1.]),
   'num_detections': 4},
  {'detection_boxes': array([[0.42056396, 0.60360718, 0.42693604, 0.61279907],
          [0.50881348, 0.53591614, 0.52868652, 0.54377136],
          [0.51214844, 0.52104492, 0.56160156, 0.56176758],
          [0.38707717, 0.        , 0.45749426, 1.        ]]),
   'detection_scores': array([0.62597656, 0.23095703, 0.8125    , 0.27319533]),
   'detection_classes': array([ 2.,  1.,  4., -1.]),
   'num_detections': 4}],
 [{'detection_boxes': array([[0.25732422, 0.48378372, 0.26416016, 0.49766159],
          [0.26962891, 0.95263672, 0.27802734, 0.96337891],
          [0.2402771 ,

In [15]:
batch_outputs = dan.detect((image_rgb, image_ir), output_mode="qa")
for n, batch_preds in enumerate(batch_outputs):
    print(f"-- Output {n}")
    for i, sample_preds in enumerate(batch_preds):
        print(f"Sample {i}:")
        for bbox, label, cls, score in sample_preds:
            print(f"{bbox=}, {label=}, {cls=}, {score=}")

-- Output 0
Sample 0:
bbox=array([0.60360718, 0.42056396, 0.61279907, 0.42693604]), label='BOAT', cls=1, score=0.6259765625
bbox=array([0.53591614, 0.50881348, 0.54377136, 0.52868652]), label='HUMAN', cls=7, score=0.23095703125
bbox=array([0.52104492, 0.51214844, 0.56176758, 0.56160156]), label='MOTORBOAT', cls=9, score=0.8125
bbox=array([0.        , 0.38707717, 1.        , 0.45749426]), label='HORIZON', cls=6, score=0.2731953263282776
Sample 1:
bbox=array([0.60360718, 0.42056396, 0.61279907, 0.42693604]), label='BOAT', cls=1, score=0.6259765625
bbox=array([0.53591614, 0.50881348, 0.54377136, 0.52868652]), label='HUMAN', cls=7, score=0.23095703125
bbox=array([0.52104492, 0.51214844, 0.56176758, 0.56160156]), label='MOTORBOAT', cls=9, score=0.8125
bbox=array([0.        , 0.38707717, 1.        , 0.45749426]), label='HORIZON', cls=6, score=0.2731953263282776
-- Output 1
Sample 0:
bbox=array([0.48378372, 0.25732422, 0.49766159, 0.26416016]), label='FAR_AWAY_OBJECT', cls=4, score=0.56396484